Universidade Tecnológica Federal do Paraná, Câmpus Campo Mourão - UTFPR-CM

Departamento Acadêmico de Computação - DACOM

BCC6003 - Inteligência Computacional

Prof. Dr. Juliano Henrique Foleis

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

In [2]:
X, y = load_wine(return_X_y=True)

In [3]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, stratify=y, test_size=0.2,
                                                        random_state=99)

# Separar o conjunto de treino em treino e validação. 
# O conjunto de validação é usado para avaliar o desempenho do modelo no processo de otimização de hiperparametros.
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.2,
                                                    random_state=101, stratify=y_treino)

# Padronizar a escala
ss = StandardScaler()
# As estatísticas para a padronização são calculadas apenas com os dados de treinamento
X_treino = ss.fit_transform(X_treino)

# Padronizar a escala do conjunto de validação de acordo com as estatísticas do conjunto de treinamento
X_val = ss.transform(X_val)

# No conjunto de testes a padronização é aplicada de acordo com as estatísticas do conjunto de treinamento.
X_teste = ss.transform(X_teste)

In [4]:
# Lista com todas as combinações de hiperparâmetros avaliadas
parametros = []
# Lista com a acurácia de todas as combinações de hiperparâmetros avaliadas (no cj. de validação)
resultados_val = []
for k in [1, 3, 5, 7, 9]:
    for md in ["euclidean", "manhattan"]:   # md: métrica de distância
        # Salvar a combinação atual de parâmetros na lista
        parametros.append((k, md))
        # Instanciar e treinar um KNN com a combinação atual de parâmetros
        knn = KNeighborsClassifier(n_neighbors=k, metric=md)
        knn.fit(X_treino, y_treino)
        # Realizar a predição no conjunto de validação
        pred_val = knn.predict(X_val)
        # Calcular a acurácia no conjunto de validação
        acc_val = accuracy_score(y_val, pred_val)
        # salvar a acurácia na lista de resultados
        resultados_val.append(acc_val)
        print("Combinação %s: %.2f" % (parametros[-1], resultados_val[-1]))

# Pegar a posição com a maior acurácia
idx_maior = np.argmax(resultados_val)
# Pegar os parâmetros correspondentes a maior acurácia na lista dos parametros
melhores_parametros = parametros[idx_maior]

# combinar treino e validação em um único conjunto
X_treino2 = np.vstack((X_treino, X_val))
y_treino2 = np.concatenate((y_treino, y_val))

#Extrair a melhor combinação de k e métrica de distância
melhor_k, melhor_md = melhores_parametros
print("Melhor combinação de parâmetros: k=%d, métrica=%s" % (melhor_k, melhor_md))

# Treinar o classificador com a melhor combinação!
knn = KNeighborsClassifier(n_neighbors=melhor_k, metric=melhor_md)
knn.fit(X_treino2, y_treino2)

Combinação (1, 'euclidean'): 0.86
Combinação (1, 'manhattan'): 0.90
Combinação (3, 'euclidean'): 0.86
Combinação (3, 'manhattan'): 0.90
Combinação (5, 'euclidean'): 0.90
Combinação (5, 'manhattan'): 0.90
Combinação (7, 'euclidean'): 0.93
Combinação (7, 'manhattan'): 0.93
Combinação (9, 'euclidean'): 0.90
Combinação (9, 'manhattan'): 0.90
Melhor combinação de parâmetros: k=7, métrica=euclidean


KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [5]:
# realizar as predições no conjunto de testes
pred = knn.predict(X_teste)
# mostrar a acurácia no conjunto de testes
print("Acurácia no teste: %.2f" % (accuracy_score(y_teste, pred)))

Acurácia no teste: 0.94


# Otimização de Hiperparâmetros (do KNN) em uma Função

In [6]:
def otimizar_knn(X_treino, y_treino, X_val, y_val, ks, mds, mensagens=False):
    # Lista com todas as combinações de hiperparâmetros avaliadas
    parametros = []
    # Lista com a acurácia de todas as combinações de hiperparâmetros avaliadas (no cj. de validação)
    resultados_val = []
    for k in [1, 3, 5, 7, 9]:
        for md in ["euclidean", "manhattan"]:   # md: métrica de distância
            # Salvar a combinação atual de parâmetros na lista
            parametros.append((k, md))
            # Instanciar e treinar um KNN com a combinação atual de parâmetros
            knn = KNeighborsClassifier(n_neighbors=k, metric=md)
            knn.fit(X_treino, y_treino)
            # Realizar a predição no conjunto de validação
            pred_val = knn.predict(X_val)
            # Calcular a acurácia no conjunto de validação
            acc_val = accuracy_score(y_val, pred_val)
            # salvar a acurácia na lista de resultados
            resultados_val.append(acc_val)
            if mensagens:
                print("Combinação %s: %.2f" % (parametros[-1], resultados_val[-1]))

    # Pegar a posição com a maior acurácia
    idx_maior = np.argmax(resultados_val)
    # Pegar os parâmetros correspondentes a maior acurácia na lista dos parametros
    melhores_parametros = parametros[idx_maior]

    # combinar treino e validação em um único conjunto
    X_treino2 = np.vstack((X_treino, X_val))
    y_treino2 = np.concatenate((y_treino, y_val))

    #Extrair a melhor combinação de k e métrica de distância
    melhor_k, melhor_md = melhores_parametros
    if mensagens:
        print("Melhor combinação de parâmetros: k=%d, métrica=%s" % (melhor_k, melhor_md))

    # Treinar o classificador com a melhor combinação!
    knn = KNeighborsClassifier(n_neighbors=melhor_k, metric=melhor_md)
    knn.fit(X_treino2, y_treino2)

    return knn

In [7]:
#Otimizar os hiperparametros do KNN (k=1,3,5,7,9, métricas=dist.euclidiana e dist. manhattan)
knn = otimizar_knn(X_treino, y_treino, X_val, y_val, 
                   ks=range(1,10,2), 
                   mds=["euclidean", "manhattan"], 
                   mensagens=False)
pred = knn.predict(X_teste)
acc = accuracy_score(y_teste, pred)
print("Acurácia no conjunto de testes: %.2f" % (acc))

Acurácia no conjunto de testes: 0.94
